<a href="https://colab.research.google.com/github/textspur/prompting_open_llm/blob/main/Example_FirstStep_zephyr_7b_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Steps covered:

*   Installation: Instructions to install necessary libraries (transformers and accelerate) from Hugging Face.
*   Load Dependencies: Importing torch and the pipeline from transformers.
*   Model Loading: Initializing the text generation pipeline with the Zephyr-7b-alpha model.
*   First Prompt Setup: Demonstrates how to set up a prompt for text generation, including defining roles (system, user, assistant) for the interaction.
* Response Generation: Shows how to generate a response from the model based on the provided prompt.
* Functions for Text Classification: Includes functions for zero-shot and one-shot text classification, demonstrating the versatility of the model.
* Example Runs: Provides example runs for both zero-shot and few-shot text classification using the model.



## Install

In [1]:
%pip install git+https://github.com/huggingface/transformers.git
%pip install accelerate

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-hnr85yvv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hnr85yvv
  Resolved https://github.com/huggingface/transformers.git to commit acd653164b6874e395ac9d46850f67599d8cdb58
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Load dependencies

In [2]:
import torch
from transformers import pipeline

# Load model

In [3]:
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

## First Prompt

In [4]:
# Each message can have 1 of 3 roles: "system" (to provide initial instructions), "user", or "assistant". For inference, make sure "user" is the role in the final message.
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate.",
    },
    {
        "role": "user",
        "content": "How many helicopters can a human eat in one sitting?"},
]

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>



Generate a response (took 47 seconds):

In [7]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1517: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Matey, I'm afraid no human can eat a helicopter in one sitting. Helicopters are not edible, and they're too large to fit in the human digestive system. Ye be hearin' me, me hearties!


Return only generated assistant message, since we are not using a seed a different answer is given:

In [11]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, return_full_text=False)
print(outputs[0]["generated_text"])

Ahoy there, matey! As for your question, I'm afraid none, for a human can't eat a helicopter, and even if they could, they wouldn't be able to consume one in a single sitting! Yarr!


# Functions



```
{"role": "system", "content":"Text of System Prompt"},
{"role": "user", "content": "Question and text to classify"},
```



## Zero-shot prompt messages

In [12]:
def zero_shot_prompt_messages(system_prompt, comment_text, question):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": 'Text to classify: ' + str(comment_text) + ' \n '   +  str(question)},
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt


In [18]:
system_prompt = "You are a reliabel chat assistant for classifying tweets for their sentiment."
question = "Is the sentiment for this tweet positive, negative or neutral (Respond only with:  ['positive', 'neutral', 'negative'])?"

textToClassify = "The weather is very good! I love it"

first_zero_shot_prompt = zero_shot_prompt_messages(system_prompt, textToClassify, question)
print(first_zero_shot_prompt)

<|system|>
You are a reliabel chat assistant for classifying tweets for their sentiment.</s>
<|user|>
Text to classify: The weather is very good! I love it 
 Is the sentiment for this tweet positive, negative or neutral (Respond only with:  ['positive', 'neutral', 'negative'])?</s>
<|assistant|>



Run text classification example

In [19]:
outputs = pipe(first_zero_shot_prompt, max_new_tokens=10, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, return_full_text=False)
print(outputs[0]["generated_text"])

['positive']


## One or few shot prompt messages

In [20]:
def few_shot_prompt_messages(system_prompt, comment_text, question, few_shot_examples):
    messages = []
    messages.append({"role": "system", "content": system_prompt})

    for user_prompt, assistent_prompt in few_shot_examples:
        messages.append({"role": "user", "content": user_prompt + " " + question})
        messages.append({"role": "assistant", "content": assistent_prompt})

    messages.append({"role": "user", "content":  str(comment_text) + str(question) })

    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

An example using one-shot:

In [22]:
example_p = [("Missing  Foo Fighters concert on the 12th at the gorge though so that's a major bummer",  "negative")]
system_prompt = "Please help me to classify tweets for their sentiment.  "
question = "--- Is the sentiment for this tweet positive, negative or neutral (only responde with ['positive', 'neutral', 'negative'])?  "

textToClassify = "The weather is very good! I love it"


first_one_shot_prompt = few_shot_prompt_messages(system_prompt, textToClassify, question, example_p)
print(first_one_shot_prompt)

<|system|>
Please help me to classify tweets for their sentiment.  </s>
<|user|>
Missing  Foo Fighters concert on the 12th at the gorge though so that's a major bummer --- Is the sentiment for this tweet positive, negative or neutral (only responde with ['positive', 'neutral', 'negative'])?  </s>
<|assistant|>
negative</s>
<|user|>
The weather is very good! I love it--- Is the sentiment for this tweet positive, negative or neutral (only responde with ['positive', 'neutral', 'negative'])?  </s>
<|assistant|>



Run text classification example

In [23]:
outputs = pipe(first_one_shot_prompt, max_new_tokens=10, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, return_full_text=False)
print(outputs[0]["generated_text"])

positive
